## **1.Init**

In [ ]:
import json

def read_file(json_file_path):
    with open(json_file_path, 'r') as json_file:
        read_list = json.load(json_file)

    return read_list

In [ ]:
pairs = read_file("more_ooc_nooc_pairs.json")

## **2. Generate Image by Stable Diffusion**

In [ ]:
!pip install transformers torch diffusers accelerate

In [ ]:
from transformers import CLIPModel, CLIPProcessor
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

### **Stable Diffusion**

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "nitrosocke/Ghibli-Diffusion",
    torch_dtype=torch.float16,
    use_safetensors=True,
    safety_checker = None,
    requires_safety_checker = False).to(device)

In [ ]:
from torchvision import transforms
from PIL import Image

generator = torch.Generator(device=device).manual_seed(1024)

for i, pair in enumerate(pairs):
    index = pair[0]
    original_image = Image.open(pair[1])
    caption = pair[2]
    label = pair[3]
    id += 1

    generate_image = pipe(prompt=caption, image=original_image, strength=0.75, guidance_scale=7.5, generator=generator).images[0]

    output_folder = "generate" # TODO: Change the output file if needed
    os.makedirs(output_folder, exist_ok=True)
    image_path = os.path.join(output_folder, f"{index}_generated_image_{id}.png")

    generate_image.save(image_path)
    pair.append(image_path)

In [ ]:
# TODO: Specify the path where you want to save the JSON file
json_file_path = "sd_pairs.json"

with open(json_file_path, 'w') as json_file:
    json.dump(pairs, json_file)

print(f"Pairs saved to {json_file_path}")